In [ ]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from black_scholes import call_value, put_value, call_delta, put_delta, call_vega, put_vega
%matplotlib inline

In [ ]:
def read_data(filename):
    df = pd.read_csv(filename, index_col=0)

    time_to_expiry = df.filter(like='TimeToExpiry')

    stock = df.filter(like='Stock')
    stock.columns = [stock.columns.str[-5:], stock.columns.str[:-6]]

    options = pd.concat((df.filter(like='-P'), df.filter(like='-C')), axis=1)
    options.columns = [options.columns.str[-3:], options.columns.str[:-4]]

    market_data = pd.concat((stock, options), axis=1)

    return time_to_expiry, market_data

In [ ]:
# Read the market data
filename = 'Options Arbitrage.csv'
time_to_expiry, market_data = read_data(filename)

In [ ]:
# Get a list of all instrument names including the stock, and of the options only
instrument_names = list(market_data.columns.get_level_values(0).unique())
print(instrument_names)

option_names = instrument_names[1:]
print(option_names)

In [ ]:
# Add time_to_expiry to market_data
market_data['TTE'] = time_to_expiry['TimeToExpiry']

# Store timestamp in variable to prevent
# errors with multiplications and such
timestamp = market_data.index

# Set the Time to Expiry as Index
market_data = market_data.set_index('TTE')

In [ ]:
# Create Empty Dictionaries
option_values = {}
option_deltas = {}

# Set known attributes
r = 0
sigma = 0.20

# Precompute Ask and Bid Prices to avoid redundant access
ask_prices = market_data['Stock', 'AskPrice'].values
bid_prices = market_data['Stock', 'BidPrice'].values

# Get a list of unique timestamps from market_data
timestamps = market_data.index.unique()

# Function to calculate option values and deltas
def calculate_values_and_deltas(option, K, option_type, price_type, delta_multiplier):
    values = []
    deltas = []
    for i, time in enumerate(timestamps):
        if 'Call' in option_type:
            value_function = call_value
            delta_function = call_delta
        else:
            value_function = put_value
            delta_function = put_delta
        
        if price_type == 'AskPrice':
            price = ask_prices[i]
        else:
            price=bid_prices[i]
        value = value_function(price, K, time, r, sigma)
        delta = delta_function(price, K, time, r, sigma) * delta_multiplier

        values.append(value)
        deltas.append(delta)
    
    option_values[option_type, option] = values
    option_deltas[option_type, option] = deltas

# Loop through option names and calculate values and deltas
for option in option_names:
    K = int(option[-2:])
    
    if 'C' in option:
        calculate_values_and_deltas(option, K, 'Long Call', 'BidPrice', 1)
        calculate_values_and_deltas(option, K, 'Short Call', 'AskPrice', -1)
    else:
        calculate_values_and_deltas(option, K, 'Long Put', 'AskPrice', 1)
        calculate_values_and_deltas(option, K, 'Short Put', 'BidPrice', -1)

# Create DataFrames with index market_data
option_values = pd.DataFrame(option_values, index=market_data.index)
option_deltas = pd.DataFrame(option_deltas, index=market_data.index)
option_values = option_values.reindex(sorted(option_values.columns), axis=1)
option_deltas = option_deltas.reindex(sorted(option_deltas.columns), axis=1)
option_values = round(option_values, 2)

In [ ]:
# Mapping for option types to their respective long and short descriptions
option_type_mapping = {
    "C": {"Short": "Short Call", "Long": "Long Call"},
    "P": {"Short": "Short Put", "Long": "Long Put"}
}

# Loop through each option and populate market_data DataFrame with expected prices and deltas
for option in option_names:
    option_key = "C" if "C" in option else "P"
    mapping = option_type_mapping[option_key]

    market_data[option, 'Expected AskPrice'] = option_values[mapping['Short'], option]
    market_data[option, 'Expected BidPrice'] = option_values[mapping['Long'], option]
    market_data[option, 'Delta Short'] = option_deltas[mapping['Short'], option].values
    market_data[option, 'Delta Long'] = option_deltas[mapping['Long'], option].values